In [1]:
import os
!git clone https://github.com/maonilino/DeepSymReg.git
os.chdir("DeepSymReg")
print(os.getcwd())
!git checkout dependabot/pip/tensorflow-gpu-2.9.3
!pip install -r requirements.txt

Cloning into 'DeepSymReg'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 78 (delta 19), reused 26 (delta 2), pack-reused 22
Unpacking objects: 100% (78/78), done.
/content/DeepSymReg
Branch 'dependabot/pip/tensorflow-gpu-2.9.3' set up to track remote branch 'dependabot/pip/tensorflow-gpu-2.9.3' from 'origin'.
Switched to a new branch 'dependabot/pip/tensorflow-gpu-2.9.3'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.7 MB/s 
     |████████████████████████████████| 37.8 MB 162 kB/s 
     |████████████████████████████████| 16.8 MB 301 kB/s 
     |████████████████████████████████| 5.8 MB 61.7 MB/s 
     |████████████████████████████████| 511.8 MB 23 kB/s 
  Created wheel for matplotlib: filename=matplotlib-3.1.1-cp38-cp38-linux_x86_64.whl size=11152207 sha256=85afd4afa16cd6bf5da7d2c0cbd1

In [1]:
!ls

benchmark_l0.py     kinematics_sr_l0.py  README.md	   sho_sr_l0.py
benchmark.py	    kinematics_sr.py	 requirements.txt  sho_sr.py
DeepSymReg.ipynb    mnist_math.py	 results	   utils
kinematics_data.py  mnist_math_relu.py	 sho_data.py
kinematics_relu.py  __pycache__		 sho_relu.py


In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from utils import functions, pretty_print
from utils.symbolic_network import SymbolicNetL0
from utils.regularization import l12_smooth

funcs = functions.default_func
x_dim = 10
# Random data for a simple function
# x = np.random.rand(100, x_dim) * 30
# random data points between 0 and 30 with x_dim dimensions
x = np.random.rand(100, x_dim) * 2 - 1
y = (x.sum(1)) / x_dim
y = y.reshape(-1, 1)

loss_list = []  # Total loss (MSE + regularization)
error_list = []     # MSE
reg_list = []       # Regularization

# Set up TensorFlow graph for the EQL network
x_placeholder = tf.placeholder(shape=(None, x_dim), dtype=tf.float32)
sym = SymbolicNetL0(symbolic_depth=2, funcs=funcs, init_stddev=0.5)
y_hat = sym(x_placeholder)

# Set up loss function with L0.5 loss
mse = tf.losses.mean_squared_error(labels=y, predictions=y_hat)
reg_loss = sym.get_loss()
loss = mse + 1e-2 * reg_loss

# Set up TensorFlow graph for training
opt = tf.train.RMSPropOptimizer(learning_rate=1e-2)
train = opt.minimize(loss)

# Training
loss_val = np.nan
# Restart training if loss goes to NaN (which happens when gradients blow up)
while np.isnan(loss_val):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # 1st stage of training with oscillating regularization weight
        for i in range(9000):
            feed_dict = {x_placeholder: x}
            _ = sess.run(train, feed_dict=feed_dict)
            if i % 1000 == 0:
                loss_val, mse_val, reg_val, = sess.run((loss,mse,reg_loss), feed_dict=feed_dict)
                print("Epoch: %d\tTotal training loss: %f" % (i, loss_val))
                loss_list.append(loss_val)
                error_list.append(mse_val)
                reg_list.append(reg_val)
                if np.isnan(loss_val):  # If loss goes to NaN, restart training
                    break



2022-12-14 21:59:13.436512: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-12-14 21:59:15.642341: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:15.661320: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:15.661498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:15.662321: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

Epoch: 0	Total training loss: nan
Epoch: 0	Total training loss: 3320.956299
Epoch: 1000	Total training loss: nan


2022-12-14 21:59:26.812506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:26.813663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:26.814345: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:26.815114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 21:59:26.815741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

...

2022-12-14 22:01:14.551970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:14.552702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:14.553006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:14.553323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:14.553568: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Epoch: 0	Total training loss: 10276.250977
Epoch: 1000	Total training loss: nan


2022-12-14 22:01:25.733280: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:25.733802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:25.734138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:25.734567: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 22:01:25.734914: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Epoch: 0	Total training loss: 491.486786


KeyboardInterrupt: 